<center><h1>Efficient K-NN for Playlist Continuation</h1></center>
<hr>

Notebook for Article Review. Article *Efficient K-NN for Playlist Continuation* is the article for RecSys Challenge 2018. It is 9th ranking model, using K-NN based Collaborative Filtering Recommendation System.

### Thanks to...

**Question Asked**  
> [numpy.where - how to change value in numpy array by condition](https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html)  

<hr>

### Table of Contents <a id="top"></a>

1. [Make Sample Data](#1)
2. [Normal Relevance Score](#2)

<hr>

In [1]:
import numpy as np

## 1. Make Sample Data <a id="1"></a>
<p style="text-align:right;"><a href="#top">top 🔝</a><p>

In [2]:
N_PLAYLIST_TR = 100
N_PLAYLIST_TS = 10
N_TRACK = 50

In [3]:
def make_playlist(n=N_PLAYLIST_TR, mode='train'):
    playlists = ['playlist-{}-{}'.format(mode, idx) for idx in range(n)]
    idx2playlist = {idx:playlist for idx, playlist in enumerate(playlists)}
    playlist2idx = {playlist:idx for idx, playlist in enumerate(playlists)}
    
    return playlists, idx2playlist, playlist2idx

In [4]:
playlists_tr, idx2playlist_tr, playlist2idx_tr = make_playlist()
for idx, playlist in idx2playlist_tr.items():
    print('{:2d}\t{}'.format(idx, playlist))

 0	playlist-train-0
 1	playlist-train-1
 2	playlist-train-2
 3	playlist-train-3
 4	playlist-train-4
 5	playlist-train-5
 6	playlist-train-6
 7	playlist-train-7
 8	playlist-train-8
 9	playlist-train-9
10	playlist-train-10
11	playlist-train-11
12	playlist-train-12
13	playlist-train-13
14	playlist-train-14
15	playlist-train-15
16	playlist-train-16
17	playlist-train-17
18	playlist-train-18
19	playlist-train-19
20	playlist-train-20
21	playlist-train-21
22	playlist-train-22
23	playlist-train-23
24	playlist-train-24
25	playlist-train-25
26	playlist-train-26
27	playlist-train-27
28	playlist-train-28
29	playlist-train-29
30	playlist-train-30
31	playlist-train-31
32	playlist-train-32
33	playlist-train-33
34	playlist-train-34
35	playlist-train-35
36	playlist-train-36
37	playlist-train-37
38	playlist-train-38
39	playlist-train-39
40	playlist-train-40
41	playlist-train-41
42	playlist-train-42
43	playlist-train-43
44	playlist-train-44
45	playlist-train-45
46	playlist-train-46
47	playlist-train-47
48

In [5]:
playlists_ts, idx2playlist_ts, playlist2idx_ts = make_playlist(n=N_PLAYLIST_TS, mode='test')
for idx, playlist in idx2playlist_ts.items():
    print('{:2d}\t{}'.format(idx, playlist))

 0	playlist-test-0
 1	playlist-test-1
 2	playlist-test-2
 3	playlist-test-3
 4	playlist-test-4
 5	playlist-test-5
 6	playlist-test-6
 7	playlist-test-7
 8	playlist-test-8
 9	playlist-test-9


In [6]:
def make_track(n=N_TRACK):
    tracks = ['track-{}'.format(idx) for idx in range(n)]
    idx2track = {idx:track for idx, track in enumerate(tracks)}
    track2idx = {track:idx for idx, track in enumerate(tracks)}
    
    return tracks, idx2track, track2idx

In [7]:
tracks, idx2track, track2idx = make_track()
for idx, track in idx2track.items():
    print('{:2d}\t{}'.format(idx, track))

 0	track-0
 1	track-1
 2	track-2
 3	track-3
 4	track-4
 5	track-5
 6	track-6
 7	track-7
 8	track-8
 9	track-9
10	track-10
11	track-11
12	track-12
13	track-13
14	track-14
15	track-15
16	track-16
17	track-17
18	track-18
19	track-19
20	track-20
21	track-21
22	track-22
23	track-23
24	track-24
25	track-25
26	track-26
27	track-27
28	track-28
29	track-29
30	track-30
31	track-31
32	track-32
33	track-33
34	track-34
35	track-35
36	track-36
37	track-37
38	track-38
39	track-39
40	track-40
41	track-41
42	track-42
43	track-43
44	track-44
45	track-45
46	track-46
47	track-47
48	track-48
49	track-49


In [8]:
def make_data(min_track=5, max_track=10, train=True):
    playlists = playlists_tr if train else playlists_ts
    playlist2idx = playlist2idx_tr if train else playlist2idx_ts
    
    data_str = {playlist: list() for playlist in playlists}
    data_idx = [[] for _ in range(len(playlists))]
    
    for playlist in playlists:
        length = np.random.randint(min_track, max_track)
        pdx = playlist2idx[playlist]

        for _ in range(length):
            tdx = np.random.randint(0, N_TRACK - 1)
            track = idx2track[tdx]

            data_str[playlist].append(track)
            data_idx[pdx].append(tdx)
    
    return data_str, data_idx

In [9]:
train_data_str, train_data_idx = make_data()

In [10]:
train_data_str

{'playlist-train-0': ['track-1',
  'track-18',
  'track-22',
  'track-47',
  'track-3',
  'track-18',
  'track-19'],
 'playlist-train-1': ['track-13',
  'track-40',
  'track-1',
  'track-0',
  'track-41',
  'track-45',
  'track-36'],
 'playlist-train-2': ['track-4',
  'track-24',
  'track-36',
  'track-37',
  'track-22',
  'track-19',
  'track-40',
  'track-23'],
 'playlist-train-3': ['track-20',
  'track-12',
  'track-14',
  'track-46',
  'track-27',
  'track-26',
  'track-15'],
 'playlist-train-4': ['track-7',
  'track-44',
  'track-2',
  'track-16',
  'track-15',
  'track-4',
  'track-28',
  'track-35'],
 'playlist-train-5': ['track-43',
  'track-43',
  'track-25',
  'track-22',
  'track-40',
  'track-18',
  'track-42',
  'track-46',
  'track-2'],
 'playlist-train-6': ['track-16',
  'track-22',
  'track-17',
  'track-8',
  'track-3',
  'track-27'],
 'playlist-train-7': ['track-48',
  'track-15',
  'track-25',
  'track-45',
  'track-23',
  'track-5'],
 'playlist-train-8': ['track-39'

In [11]:
train_data_idx

[[1, 18, 22, 47, 3, 18, 19],
 [13, 40, 1, 0, 41, 45, 36],
 [4, 24, 36, 37, 22, 19, 40, 23],
 [20, 12, 14, 46, 27, 26, 15],
 [7, 44, 2, 16, 15, 4, 28, 35],
 [43, 43, 25, 22, 40, 18, 42, 46, 2],
 [16, 22, 17, 8, 3, 27],
 [48, 15, 25, 45, 23, 5],
 [39, 48, 29, 46, 48, 2, 2, 9],
 [46, 9, 2, 34, 23, 35],
 [10, 46, 11, 43, 16, 33],
 [36, 0, 11, 38, 43, 48, 14, 31],
 [34, 26, 26, 22, 18, 41],
 [29, 17, 20, 48, 2, 36, 18, 5, 27],
 [37, 38, 8, 15, 13, 10, 0, 44],
 [3, 26, 2, 14, 14, 2, 38],
 [30, 5, 15, 0, 2, 33, 30],
 [13, 17, 13, 7, 21],
 [16, 36, 36, 6, 20],
 [5, 34, 22, 32, 24],
 [21, 14, 25, 3, 18, 29, 41],
 [5, 36, 14, 35, 25, 31, 36, 13],
 [3, 41, 24, 28, 3, 20, 4, 44, 17],
 [1, 40, 45, 29, 20, 6, 6, 25],
 [6, 28, 14, 47, 5, 8, 30, 24, 9],
 [8, 41, 44, 48, 5, 41, 10],
 [26, 17, 31, 18, 3, 16],
 [25, 42, 39, 37, 21, 32, 34, 21, 27],
 [23, 19, 4, 33, 10, 29],
 [22, 25, 24, 19, 28, 23, 44, 5, 25],
 [30, 4, 46, 31, 1, 10, 28, 37],
 [4, 3, 48, 17, 23, 19, 7, 36],
 [23, 29, 28, 40, 12, 40, 38,

In [12]:
test_data_str, test_data_idx = make_data(min_track=1, max_track=5, train=False)

In [13]:
test_data_str

{'playlist-test-0': ['track-15', 'track-35', 'track-13', 'track-44'],
 'playlist-test-1': ['track-46'],
 'playlist-test-2': ['track-37', 'track-14'],
 'playlist-test-3': ['track-21'],
 'playlist-test-4': ['track-46'],
 'playlist-test-5': ['track-23', 'track-48', 'track-39'],
 'playlist-test-6': ['track-31', 'track-44', 'track-3'],
 'playlist-test-7': ['track-48', 'track-45', 'track-17'],
 'playlist-test-8': ['track-42', 'track-24', 'track-13', 'track-1'],
 'playlist-test-9': ['track-16', 'track-24']}

In [14]:
test_data_idx

[[15, 35, 13, 44],
 [46],
 [37, 14],
 [21],
 [46],
 [23, 48, 39],
 [31, 44, 3],
 [48, 45, 17],
 [42, 24, 13, 1],
 [16, 24]]

In [15]:
def to_ptm(data, ignore_frequency=True):
    row = len(data)
    col = N_TRACK
    
    ptm = np.zeros((row, col))
    
    for pdx, tracks in enumerate(data):
        for tdx in tracks:
            if ignore_frequency:
                ptm[pdx, tdx] = 1
            else:
                ptm[pdx, tdx] += 1
    
    return ptm

In [16]:
train_ptm = to_ptm(train_data_idx)

In [17]:
train_ptm.shape

(100, 50)

In [18]:
train_ptm

array([[0., 1., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [19]:
test_ptm = to_ptm(test_data_idx)

In [20]:
test_ptm.shape

(10, 50)

In [21]:
test_ptm

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

## 2. Normal Relevance Score <a id="2"></a>
<p style="text-align: right;"><a href="#top">top 🔝</a></p>

<br>**Playlist Similarity**<br><br>
> Calculate Playlist Similarity based on Cosine Similarity

In [157]:
def cosine_similarity(test, train, weight=None):
    denominator = np.sqrt(np.sum(test * test)) * np.sqrt(np.sum(train * train))
    numerator = test * train
    
    similarity = numerator / denominator
    
    if weight.any():
        similarity = similarity * weight

    return np.sum(similarity)

In [23]:
'''
Test Cosine Similarity
'''
a = np.array([1, 1, 1, 1])
b = np.array([1, 1, 1, 1])
cosine_similarity(a, b)

1.0

In [24]:
'''
Test Cosine Similarity
'''
a = np.array([1, 0, 1, 1])
b = np.array([0, 1, 0, 0])
cosine_similarity(a, b)

0.0

In [166]:
def calculate_similarity(test_ptm, train_ptm, idf=False, rho=0.4):
    row = N_PLAYLIST_TS  # tset_ptm.shape[0]
    col = N_PLAYLIST_TR  # train_ptm.shape[0]

    similarity = np.zeros((row, col))

    if idf:
        weight = train_ptm.sum(axis=0)
        weight = weight.astype(np.complex)
        weight = 1.0 / (((weight - 1) ** rho) + 1)
        weight = weight.astype(np.float64)
    
    for ts in range(N_PLAYLIST_TS):
        test = test_ptm[ts, :]
        for tr in range(N_PLAYLIST_TR):
            train = train_ptm[tr, :]
                        
            similarity[ts, tr] = cosine_similarity(test, train, weight)
        
    return similarity

In [131]:
p2p_similarity = calculate_similarity(test_ptm, train_ptm)

In [132]:
p2p_similarity.shape

(10, 100)

In [133]:
p2p_similarity

array([[0.        , 0.18898224, 0.        , 0.18898224, 0.53033009,
        0.        , 0.        , 0.20412415, 0.        , 0.20412415,
        0.        , 0.        , 0.        , 0.        , 0.53033009,
        0.        , 0.20412415, 0.25      , 0.        , 0.        ,
        0.        , 0.37796447, 0.1767767 , 0.        , 0.        ,
        0.20412415, 0.        , 0.        , 0.        , 0.1767767 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.1767767 , 0.        , 0.        , 0.        , 0.20412415,
        0.        , 0.        , 0.        , 0.        , 0.1767767 ,
        0.        , 0.18898224, 0.16666667, 0.2236068 , 0.        ,
        0.35355339, 0.        , 0.20412415, 0.        , 0.20412415,
        0.        , 0.2236068 , 0.2236068 , 0.18898224, 0.37796447,
        0.        , 0.        , 0.        , 0.20412415, 0.        ,
        0.        , 0.        , 0.2236068 , 0.28867513, 0.        ,
        0.        , 0.        , 0.2236068 , 0.18

In [167]:
p2p_similarity_idf = calculate_similarity(test_ptm, train_ptm, idf=True)

/Users/dhsong/Workspaces/Practices/Playlist Continuation/venv/lib/python3.7/site-packages/ipykernel_launcher.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  # This is added back by InteractiveShellApp.init_path()


In [168]:
p2p_similarity_idf.shape

(10, 100)

In [169]:
p2p_similarity_idf

array([[0.        , 0.05381217, 0.        , 0.04878516, 0.14699803,
        0.        , 0.        , 0.052694  , 0.        , 0.06190464,
        0.        , 0.        , 0.        , 0.        , 0.14372372,
        0.        , 0.052694  , 0.07118681, 0.        , 0.        ,
        0.        , 0.11112473, 0.0477527 , 0.        , 0.        ,
        0.05514007, 0.        , 0.        , 0.        , 0.0477527 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.04563434, 0.        , 0.        , 0.        , 0.052694  ,
        0.        , 0.        , 0.        , 0.        , 0.0477527 ,
        0.        , 0.05104979, 0.04302447, 0.06367142, 0.        ,
        0.09924533, 0.        , 0.05812379, 0.        , 0.05514007,
        0.        , 0.06781313, 0.06781313, 0.05381217, 0.10609772,
        0.        , 0.        , 0.        , 0.05812379, 0.        ,
        0.        , 0.        , 0.05772338, 0.08754638, 0.        ,
        0.        , 0.        , 0.06040292, 0.05

<br>**Relevance Score**<br><br>
> For All the Track, Calculate Relevance Score with Playlist in test

In [29]:
def find_neighbors(ts, similarity=p2p_similarity, k=5):
    neighbor_idx = similarity[ts].argsort()[::-1][:k]
    neighbor_similarity = similarity[ts, neighbor_idx]
    
    return neighbor_idx, neighbor_similarity

In [87]:
def min_max_normalize(similarity):
    numerator = similarity - similarity.min()
    denominator = similarity.max() - similarity.min()
    return numerator / denominator


In [170]:
def calculate_relevance(normalize=False, amplification=False, alpha=2, idf=False):
    row = N_PLAYLIST_TS
    col = N_TRACK
    
    relevance = np.zeros((row, col))
    
    for ts in range(N_PLAYLIST_TS):
        if idf:
            neighbor_idx, neighbor_similarity = find_neighbors(ts, similarity=p2p_similarity_idf)
        else:
            neighbor_idx, neighbor_similarity = find_neighbors(ts)
        
        if normalize:
            neighbor_similarity = min_max_normalize(neighbor_similarity)
        if amplification:
            neighbor_similarity = np.square(neighbor_similarity)
        
        for t in range(N_TRACK):
            denominator = np.sum(neighbor_similarity)
            numerator = np.sum(neighbor_similarity * train_ptm[neighbor_idx, t])
            
            relevance[ts, t] = numerator / denominator
    return relevance

In [79]:
relevance = calculate_relevance()

In [80]:
relevance.shape

(10, 50)

In [81]:
relevance

array([[0.42186381, 0.        , 0.23836802, 0.35337987, 0.23836802,
        0.33976818, 0.        , 0.23836802, 0.23836802, 0.        ,
        0.23836802, 0.        , 0.        , 0.59174789, 0.16988409,
        0.64662013, 0.23836802, 0.        , 0.        , 0.        ,
        0.        , 0.18349579, 0.        , 0.        , 0.        ,
        0.16988409, 0.16988409, 0.        , 0.23836802, 0.        ,
        0.        , 0.33976818, 0.        , 0.16988409, 0.18349579,
        0.57813619, 0.16988409, 0.23836802, 0.23836802, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.66023182,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.19082621, 0.19082621, 0.41223172, 0.        , 0.        ,
        0.19082621, 0.        , 0.        , 0.        , 0.60305793,
        0.20611586, 0.20611586, 0.19082621, 0.        , 0.19082621,
        0.        , 0.58776828, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.2

In [101]:
relevance_norm_amp = calculate_relevance(normalize=True, amplification=True)

In [102]:
relevance_norm_amp.shape

(10, 50)

In [103]:
relevance_norm_amp

array([[0.50968484, 0.        , 0.49031516, 0.01936969, 0.49031516,
        0.        , 0.        , 0.49031516, 0.49031516, 0.        ,
        0.49031516, 0.        , 0.        , 0.50968484, 0.        ,
        0.98063031, 0.49031516, 0.        , 0.        , 0.        ,
        0.        , 0.01936969, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.49031516, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.01936969,
        0.49031516, 0.        , 0.49031516, 0.49031516, 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.66666667, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.66666667,
        0.33333333, 0.33333333, 0.        , 0.        , 0.        ,
        0.        , 0.33333333, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.3

In [171]:
relevance_norm_amp_idf = calculate_relevance(normalize=True, amplification=True, idf=True)

In [172]:
relevance_norm_amp_idf.shape

(10, 50)

In [173]:
relevance_norm_amp_idf

array([[0.46350334, 0.        , 0.52851264, 0.01622457, 0.52851264,
        0.00798402, 0.        , 0.52851264, 0.44727877, 0.        ,
        0.44727877, 0.        , 0.        , 0.47148736, 0.00798402,
        0.97579141, 0.52851264, 0.        , 0.        , 0.        ,
        0.        , 0.01622457, 0.        , 0.        , 0.        ,
        0.00798402, 0.        , 0.        , 0.52851264, 0.        ,
        0.        , 0.00798402, 0.        , 0.        , 0.01622457,
        0.53649666, 0.00798402, 0.44727877, 0.44727877, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.99201598,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.66666667, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.66666667,
        0.33333333, 0.33333333, 0.        , 0.        , 0.        ,
        0.        , 0.33333333, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.3

<br>**Recommend Track - Score A(Basic)**<br><br>
> Recommend Track for Test Playlist

In [34]:
N_RECOMMEND = 8

In [35]:
test_data_rec_str = {playlist: list() for playlist in playlists_ts}
test_data_rec_idx = [[] for _ in range(len(playlists_ts))]

In [36]:
for idx in range(N_PLAYLIST_TS):
    recommendation = relevance[idx, :].argsort()[::-1][:N_RECOMMEND]
    test_data_rec_idx[idx] = recommendation.tolist()
    test_data_rec_str[idx2playlist_ts[idx]] = [idx2track[idx] for idx in recommendation]
    print('Current Track in Playlist {}'.format(idx2playlist_ts[idx]))
    print(test_data_str[idx2playlist_ts[idx]])
    print('Recommended Track')
    print(test_data_rec_str[idx2playlist_ts[idx]])
    print('\n-------------------------------------------\n')

Current Track in Playlist playlist-test-0
['track-15', 'track-35', 'track-13', 'track-44']
Recommended Track
['track-44', 'track-15', 'track-13', 'track-35', 'track-0', 'track-3', 'track-5', 'track-31']

-------------------------------------------

Current Track in Playlist playlist-test-1
['track-46']
Recommended Track
['track-46', 'track-9', 'track-16', 'track-2', 'track-39', 'track-35', 'track-48', 'track-23']

-------------------------------------------

Current Track in Playlist playlist-test-2
['track-37', 'track-14']
Recommended Track
['track-14', 'track-37', 'track-24', 'track-38', 'track-2', 'track-3', 'track-26', 'track-5']

-------------------------------------------

Current Track in Playlist playlist-test-3
['track-21']
Recommended Track
['track-21', 'track-13', 'track-44', 'track-3', 'track-34', 'track-7', 'track-17', 'track-25']

-------------------------------------------

Current Track in Playlist playlist-test-4
['track-46']
Recommended Track
['track-46', 'track-9', '

In [37]:
test_data_rec_str

{'playlist-test-0': ['track-44',
  'track-15',
  'track-13',
  'track-35',
  'track-0',
  'track-3',
  'track-5',
  'track-31'],
 'playlist-test-1': ['track-46',
  'track-9',
  'track-16',
  'track-2',
  'track-39',
  'track-35',
  'track-48',
  'track-23'],
 'playlist-test-2': ['track-14',
  'track-37',
  'track-24',
  'track-38',
  'track-2',
  'track-3',
  'track-26',
  'track-5'],
 'playlist-test-3': ['track-21',
  'track-13',
  'track-44',
  'track-3',
  'track-34',
  'track-7',
  'track-17',
  'track-25'],
 'playlist-test-4': ['track-46',
  'track-9',
  'track-16',
  'track-2',
  'track-39',
  'track-35',
  'track-48',
  'track-23'],
 'playlist-test-5': ['track-23',
  'track-48',
  'track-39',
  'track-9',
  'track-5',
  'track-41',
  'track-28',
  'track-15'],
 'playlist-test-6': ['track-3',
  'track-31',
  'track-44',
  'track-4',
  'track-26',
  'track-17',
  'track-30',
  'track-22'],
 'playlist-test-7': ['track-45',
  'track-48',
  'track-17',
  'track-23',
  'track-25',
  '

In [38]:
test_data_rec_idx

[[44, 15, 13, 35, 0, 3, 5, 31],
 [46, 9, 16, 2, 39, 35, 48, 23],
 [14, 37, 24, 38, 2, 3, 26, 5],
 [21, 13, 44, 3, 34, 7, 17, 25],
 [46, 9, 16, 2, 39, 35, 48, 23],
 [23, 48, 39, 9, 5, 41, 28, 15],
 [3, 31, 44, 4, 26, 17, 30, 22],
 [45, 48, 17, 23, 25, 4, 9, 13],
 [13, 41, 1, 42, 2, 45, 31, 36],
 [24, 16, 5, 20, 6, 14, 9, 1]]

In [39]:
test_data_idx

[[15, 35, 13, 44],
 [46],
 [37, 14],
 [21],
 [46],
 [23, 48, 39],
 [31, 44, 3],
 [48, 45, 17],
 [42, 24, 13, 1],
 [16, 24]]

<br>**Evaluation**<br><br>
> Evaluate the Recommendation

In [40]:
def evaluate_rprec(pred, real):
    length = [len(playlist) for playlist in real]
    
    correct = list()
    for idx, l in enumerate(length):
        counter = 0
        for track in pred[idx][:l]:
            counter += (1 if track in real[idx] else 0)
        correct.append(counter)
    
    rprec = np.mean(np.array(correct) / np.array(length))
    return rprec

In [41]:
rprec = evaluate_rprec(test_data_rec_idx, test_data_idx)
print('R Precision Evaluation: {:.4f}'.format(rprec))

R Precision Evaluation: 0.9750


In [42]:
def evaluate_ndcg(pred, real, relevance=relevance):
    length = [len(playlist) for playlist in real]
    
    ndcg = np.zeros((N_PLAYLIST_TS))
    for idx, l in enumerate(length):
        denominator = np.array([np.log2(n + 2) for n in range(l)])
        
        dcg = np.cumsum(relevance[idx, real[idx]])
        dcg = dcg / denominator

        idcg = np.cumsum(relevance[idx, pred[idx][:l]])
        idcg = idcg / denominator

        ndcg[idx] = np.mean(dcg / idcg)
    
    ndcg = np.mean(ndcg)
    return ndcg

In [43]:
ndcg = evaluate_ndcg(test_data_rec_idx, test_data_idx)
print('N-DCG Evaluation: {:.4f}'.format(ndcg))

N-DCG Evaluation: 0.9485


<hr>

<br>**Recommend Track - Score B(Normalization + Amplification)**<br><br>
> Recommend Track for Test Playlist

In [104]:
N_RECOMMEND = 8

In [105]:
test_data_rec_str_b = {playlist: list() for playlist in playlists_ts}
test_data_rec_idx_b = [[] for _ in range(len(playlists_ts))]

In [106]:
for idx in range(N_PLAYLIST_TS):
    recommendation_b = relevance_norm_amp[idx, :].argsort()[::-1][:N_RECOMMEND]
    test_data_rec_idx_b[idx] = recommendation_b.tolist()
    test_data_rec_str_b[idx2playlist_ts[idx]] = [idx2track[idx] for idx in recommendation_b]
    print('Current Track in Playlist {}'.format(idx2playlist_ts[idx]))
    print(test_data_str[idx2playlist_ts[idx]])
    print('Recommended Track')
    print(test_data_rec_str_b[idx2playlist_ts[idx]])
    print('\n-------------------------------------------\n')

Current Track in Playlist playlist-test-0
['track-15', 'track-35', 'track-13', 'track-44']
Recommended Track
['track-44', 'track-15', 'track-0', 'track-13', 'track-8', 'track-16', 'track-28', 'track-10']

-------------------------------------------

Current Track in Playlist playlist-test-1
['track-46']
Recommended Track
['track-46', 'track-2', 'track-9', 'track-35', 'track-39', 'track-10', 'track-33', 'track-34']

-------------------------------------------

Current Track in Playlist playlist-test-2
['track-37', 'track-14']
Recommended Track
['track-14', 'track-24', 'track-37', 'track-5', 'track-10', 'track-38', 'track-40', 'track-2']

-------------------------------------------

Current Track in Playlist playlist-test-3
['track-21']
Recommended Track
['track-21', 'track-17', 'track-7', 'track-13', 'track-44', 'track-25', 'track-26', 'track-40']

-------------------------------------------

Current Track in Playlist playlist-test-4
['track-46']
Recommended Track
['track-46', 'track-2'

In [107]:
test_data_rec_str_b

{'playlist-test-0': ['track-44',
  'track-15',
  'track-0',
  'track-13',
  'track-8',
  'track-16',
  'track-28',
  'track-10'],
 'playlist-test-1': ['track-46',
  'track-2',
  'track-9',
  'track-35',
  'track-39',
  'track-10',
  'track-33',
  'track-34'],
 'playlist-test-2': ['track-14',
  'track-24',
  'track-37',
  'track-5',
  'track-10',
  'track-38',
  'track-40',
  'track-2'],
 'playlist-test-3': ['track-21',
  'track-17',
  'track-7',
  'track-13',
  'track-44',
  'track-25',
  'track-26',
  'track-40'],
 'playlist-test-4': ['track-46',
  'track-2',
  'track-9',
  'track-35',
  'track-39',
  'track-10',
  'track-33',
  'track-34'],
 'playlist-test-5': ['track-48',
  'track-23',
  'track-39',
  'track-9',
  'track-25',
  'track-29',
  'track-46',
  'track-45'],
 'playlist-test-6': ['track-31',
  'track-44',
  'track-22',
  'track-4',
  'track-30',
  'track-3',
  'track-26',
  'track-0'],
 'playlist-test-7': ['track-45',
  'track-48',
  'track-13',
  'track-28',
  'track-9',
 

In [108]:
test_data_rec_idx_b

[[44, 15, 0, 13, 8, 16, 28, 10],
 [46, 2, 9, 35, 39, 10, 33, 34],
 [14, 24, 37, 5, 10, 38, 40, 2],
 [21, 17, 7, 13, 44, 25, 26, 40],
 [46, 2, 9, 35, 39, 10, 33, 34],
 [48, 23, 39, 9, 25, 29, 46, 45],
 [31, 44, 22, 4, 30, 3, 26, 0],
 [45, 48, 13, 28, 9, 23, 17, 15],
 [13, 1, 42, 41, 24, 33, 2, 5],
 [24, 16, 1, 46, 14, 5, 9, 20]]

In [109]:
test_data_idx

[[15, 35, 13, 44],
 [46],
 [37, 14],
 [21],
 [46],
 [23, 48, 39],
 [31, 44, 3],
 [48, 45, 17],
 [42, 24, 13, 1],
 [16, 24]]

<br>**Evaluation**<br><br>
> Evaluate the Recommendation

In [110]:
def evaluate_rprec(pred, real):
    length = [len(playlist) for playlist in real]
    
    correct = list()
    for idx, l in enumerate(length):
        counter = 0
        for track in pred[idx][:l]:
            counter += (1 if track in real[idx] else 0)
        correct.append(counter)
    
    rprec = np.mean(np.array(correct) / np.array(length))
    return rprec

In [111]:
rprec = evaluate_rprec(test_data_rec_idx_b, test_data_idx)
print('R Precision Evaluation: {:.4f}'.format(rprec))

R Precision Evaluation: 0.8333


In [112]:
def evaluate_ndcg(pred, real, relevance=relevance_norm_amp):
    length = [len(playlist) for playlist in real]
    
    ndcg = np.zeros((N_PLAYLIST_TS))
    for idx, l in enumerate(length):
        denominator = np.array([np.log2(n + 2) for n in range(l)])
        
        dcg = np.cumsum(relevance[idx, real[idx]])
        dcg = dcg / denominator

        idcg = np.cumsum(relevance[idx, pred[idx][:l]])
        idcg = idcg / denominator

        ndcg[idx] = np.mean(dcg / idcg)
    
    ndcg = np.mean(ndcg)
    return ndcg

In [113]:
ndcg = evaluate_ndcg(test_data_rec_idx_b, test_data_idx)
print('N-DCG Evaluation: {:.4f}'.format(ndcg))

N-DCG Evaluation: 0.9363


<br>**Recommend Track - Score C(Normalization + Amplification + IDF Weighting)**<br><br>
> Recommend Track for Test Playlist

In [174]:
N_RECOMMEND = 8

In [175]:
test_data_rec_str_c = {playlist: list() for playlist in playlists_ts}
test_data_rec_idx_c = [[] for _ in range(len(playlists_ts))]

In [176]:
for idx in range(N_PLAYLIST_TS):
    recommendation_c = relevance_norm_amp_idf[idx, :].argsort()[::-1][:N_RECOMMEND]
    test_data_rec_idx_c[idx] = recommendation_c.tolist()
    test_data_rec_str_c[idx2playlist_ts[idx]] = [idx2track[idx] for idx in recommendation_c]
    print('Current Track in Playlist {}'.format(idx2playlist_ts[idx]))
    print(test_data_str[idx2playlist_ts[idx]])
    print('Recommended Track')
    print(test_data_rec_str_c[idx2playlist_ts[idx]])
    print('\n-------------------------------------------\n')

Current Track in Playlist playlist-test-0
['track-15', 'track-35', 'track-13', 'track-44']
Recommended Track
['track-44', 'track-15', 'track-35', 'track-2', 'track-16', 'track-4', 'track-7', 'track-28']

-------------------------------------------

Current Track in Playlist playlist-test-1
['track-46']
Recommended Track
['track-46', 'track-2', 'track-9', 'track-35', 'track-39', 'track-10', 'track-33', 'track-34']

-------------------------------------------

Current Track in Playlist playlist-test-2
['track-37', 'track-14']
Recommended Track
['track-14', 'track-24', 'track-37', 'track-5', 'track-10', 'track-38', 'track-40', 'track-2']

-------------------------------------------

Current Track in Playlist playlist-test-3
['track-21']
Recommended Track
['track-21', 'track-17', 'track-7', 'track-13', 'track-44', 'track-25', 'track-26', 'track-40']

-------------------------------------------

Current Track in Playlist playlist-test-4
['track-46']
Recommended Track
['track-46', 'track-2',

In [177]:
test_data_rec_str_c

{'playlist-test-0': ['track-44',
  'track-15',
  'track-35',
  'track-2',
  'track-16',
  'track-4',
  'track-7',
  'track-28'],
 'playlist-test-1': ['track-46',
  'track-2',
  'track-9',
  'track-35',
  'track-39',
  'track-10',
  'track-33',
  'track-34'],
 'playlist-test-2': ['track-14',
  'track-24',
  'track-37',
  'track-5',
  'track-10',
  'track-38',
  'track-40',
  'track-2'],
 'playlist-test-3': ['track-21',
  'track-17',
  'track-7',
  'track-13',
  'track-44',
  'track-25',
  'track-26',
  'track-40'],
 'playlist-test-4': ['track-46',
  'track-2',
  'track-9',
  'track-35',
  'track-39',
  'track-10',
  'track-33',
  'track-34'],
 'playlist-test-5': ['track-48',
  'track-39',
  'track-9',
  'track-29',
  'track-46',
  'track-2',
  'track-23',
  'track-25'],
 'playlist-test-6': ['track-31',
  'track-44',
  'track-22',
  'track-4',
  'track-30',
  'track-3',
  'track-26',
  'track-18'],
 'playlist-test-7': ['track-45',
  'track-48',
  'track-13',
  'track-28',
  'track-9',
  

In [178]:
test_data_rec_idx_c

[[44, 15, 35, 2, 16, 4, 7, 28],
 [46, 2, 9, 35, 39, 10, 33, 34],
 [14, 24, 37, 5, 10, 38, 40, 2],
 [21, 17, 7, 13, 44, 25, 26, 40],
 [46, 2, 9, 35, 39, 10, 33, 34],
 [48, 39, 9, 29, 46, 2, 23, 25],
 [31, 44, 22, 4, 30, 3, 26, 18],
 [45, 48, 13, 28, 9, 23, 15, 25],
 [13, 41, 1, 42, 36, 40, 0, 45],
 [24, 16, 5, 1, 46, 14, 9, 20]]

In [179]:
test_data_idx

[[15, 35, 13, 44],
 [46],
 [37, 14],
 [21],
 [46],
 [23, 48, 39],
 [31, 44, 3],
 [48, 45, 17],
 [42, 24, 13, 1],
 [16, 24]]

<br>**Evaluation**<br><br>
> Evaluate the Recommendation

In [180]:
def evaluate_rprec(pred, real):
    length = [len(playlist) for playlist in real]
    
    correct = list()
    for idx, l in enumerate(length):
        counter = 0
        for track in pred[idx][:l]:
            counter += (1 if track in real[idx] else 0)
        correct.append(counter)
    
    rprec = np.mean(np.array(correct) / np.array(length))
    return rprec

In [181]:
rprec = evaluate_rprec(test_data_rec_idx_c, test_data_idx)
print('R Precision Evaluation: {:.4f}'.format(rprec))

R Precision Evaluation: 0.8000


In [182]:
def evaluate_ndcg(pred, real, relevance=relevance_norm_amp_idf):
    length = [len(playlist) for playlist in real]
    
    ndcg = np.zeros((N_PLAYLIST_TS))
    for idx, l in enumerate(length):
        denominator = np.array([np.log2(n + 2) for n in range(l)])
        
        dcg = np.cumsum(relevance[idx, real[idx]])
        dcg = dcg / denominator

        idcg = np.cumsum(relevance[idx, pred[idx][:l]])
        idcg = idcg / denominator

        ndcg[idx] = np.mean(dcg / idcg)
    
    ndcg = np.mean(ndcg)
    return ndcg

In [183]:
ndcg = evaluate_ndcg(test_data_rec_idx_c, test_data_idx)
print('N-DCG Evaluation: {:.4f}'.format(ndcg))

N-DCG Evaluation: 0.9161


<hr>